# Getting Started

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('metacri_reviews_scores.csv') 

In [ ]:
df.dtypes

In [ ]:
df.shape
# df[['genre','movie_title']] = df[['genre','movie_title']].astype(str)

In [ ]:
## user score cleaning 
tbr = ['tbd']
df['userscore'] = df['userscore'].apply(lambda x: np.nan if x in tbr else x)
df = df.dropna(axis=0, how='any')
df['userscore'] = df['userscore'].astype(float)*10


# User vs Critic Summary

In [ ]:
df[['userscore','metascore']].mean()

In [ ]:
df[['userscore','metascore']].std()

In [ ]:
df['comparison'] = df['metascore'] > df['userscore']
df['difference'] = df['metascore'] - df['userscore']
print(sum(df['comparison'])/df.shape[0])
df.sort_values('difference',ascending=False)

In [ ]:
##top 30 genre indicator 
df[['genre','movie_title']] = df[['genre','movie_title']].astype(str)

In [ ]:
df.sample(10)

In [ ]:
score_df = df[['metascore', 'userscore']]
score_df.plot.scatter('metascore', 'userscore')

# User vs Critic Genre Preference

In [ ]:
df_score = df

In [ ]:
all_genres = set()
for s in df_score['genre']:
    s = s.split(',')
    all_genres = all_genres.union(s)

In [ ]:
def genreFilter(row, genre):
    genres = set(row['genre'].split(','))
    # print(genres)
    if genre in genres:
        return True
    return False

In [ ]:
all_genres

In [ ]:
for genre in all_genres:
    new_field = 'is' + genre
    df_score[new_field] = df_score.apply(genreFilter, args = (genre, ), axis = 1)

In [ ]:
df_score[df_score['isFilm-Noir']]

In [ ]:
df_score.columns[-25:]

In [ ]:
pd.options.display.max_columns = 999

scores = []
for genre in df_score.columns[-25:]:
    scores.append([df_score[df_score[genre]].userscore.mean(),df_score[df_score[genre]].metascore.mean()])

genre_comparison = pd.DataFrame(scores)
genre_comparison.index = (df_score.columns[-25:])
genre_comparison.columns=['user','meta']
genre_comparison['difference'] = abs(genre_comparison.user - genre_comparison.meta)
genre_comparison =genre_comparison.sort_values('difference',ascending=False)
genre_comparison.index = [x[2:] for x in genre_comparison.index.tolist()]
genre_comparison.user

In [ ]:
import plotly.graph_objects as go

months = genre_comparison.index.tolist()
months

fig = go.Figure()
fig.add_trace(go.Bar(
    x=months,
    y=genre_comparison.user,
    name='userscore',
    marker_color='#7393BC'
))
fig.add_trace(go.Bar(
    x=months,
    y=genre_comparison.meta,
    name='metascore',
    marker_color='#95D845'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-80,width=1000, height=600)
fig.show()